In [ ]:
import tkinter as tk
from tkinter import messagebox
from sympy import symbols, sympify, diff, Matrix, zeros, hessian, solve
from IPython.display import display, Math

def kkt1():
    m = int(entry_variabel.get())
    var = []
    for i in range(m):
        masuk = entry_vars[i].get()
        var.append(symbols(masuk))

    fungsi = entry_fungsi.get()
    fungsi = sympify(fungsi)

    n = int(entry_kendala.get())
    bener = entry_solusi.get()

    G = 0
    H = 0
    g = []
    h = []
    var_g = []
    var_h = []
    tanda = []

    for i in range(n):
        masuk = entry_kendala_funcs[i].get()
        masuk = sympify(masuk)
        masukt = entry_kendala_tanda[i].get()
        tanda.append(masukt)
        if tanda[i] == '=':
            h.append(masuk)
            var_h.append(symbols(f'lambda{i}'))
            H += 1
        elif tanda[i] == '>=':
            g.append(-masuk)
            var_g.append(symbols(f'pi{i}'))
            G += 1
        else:
            g.append(masuk)
            var_g.append(symbols(f'pi{i}'))
            G += 1
    g = [g[i] * var_g[i] for i in range(len(g))]
    h = [h[i] * var_h[i] for i in range(len(h))]

    hesian = hessian(fungsi, var)
    kondisi = True
    gabung = var + var_g + var_h
    turun=fungsi+sum(g)+sum(h)
    display(Math(latex(turun)))
    for i in gabung :
      display(Math(latex(diff(fungsi,i))))
    if kondisi:
        h = Matrix(h)
        g = Matrix(g)
        if H == 0:
            h = zeros(1, 1)
            zero_h = zeros(len(var), 1)
        else:
            gradh = [diff(i, j) for i in h for j in var]
            gradh = Matrix(gradh).reshape(len(h), len(var)).T
            zero_h = zeros(len(var), 1)
            for i in range(len(gradh[0, :])):
                zero_h += gradh[:, i]

        if G == 0:
            g = zeros(1, 1)
            zero_g = zeros(len(var), 1)
        else:
            gradg = [diff(i, j) for i in g for j in var]
            gradg = Matrix(gradg).reshape(len(g), len(var)).T
            zero_g = zeros(len(var), 1)
            for i in range(len(gradg[0, :])):
                zero_g += gradg[:, i]

        gradf = [diff(fungsi, i) for i in var]
        gradf = Matrix(gradf)
        L = gradf + zero_h + zero_g
        L = L.col_join(g)
        L = L.col_join(h)

        for i in range(len(L)):
            display(Math(latex(L[i])))

        solusi = solve(L, gabung)
        solutions = [dict(zip(gabung, solusi)) for solusi in solusi]

        # Filter solusi jika bener == 'y'
        if bener.lower() == 'y':
            solusi_baru = []
            for sol in solutions:
                if all(sol[var_name] > 0 for var_name in var):
                    solusi_baru.append(sol)
            solutions = solusi_baru

        return fungsi, solutions, L

def display_kkt_results():
    try:
        fungsi, solutions, L = kkt1()
        messagebox.showinfo("Hasil Kondisi KKT", f"Turunan fungsi objektif: {fungsi}\n\nSolusi KKT: {solutions}\n\nMatriks Kondisi KKT: {L}")
    except Exception as e:
        messagebox.showerror("Error", f"Terjadi kesalahan: {str(e)}")

def add_variable_fields():
    global entry_vars, var_entries_frame
    var_entries_frame = tk.Frame(var_frame)
    var_entries_frame.pack()

    entry_vars = []
    var_count = int(entry_variabel.get())
    for i in range(var_count):
        label_var = tk.Label(var_entries_frame, text=f"Masukkan variabel ke-{i + 1}: ")
        label_var.pack()
        entry_var = tk.Entry(var_entries_frame, width=40)
        entry_var.pack()
        entry_vars.append(entry_var)

def add_constraint_fields():
    global entry_kendala_funcs, entry_kendala_tanda, kendala_entries_frame
    kendala_entries_frame = tk.Frame(constraint_frame)
    kendala_entries_frame.pack()

    entry_kendala_funcs = []
    entry_kendala_tanda = []
    kendala_count = int(entry_kendala.get())
    for i in range(kendala_count):
        label_kendala_func = tk.Label(kendala_entries_frame, text=f"Masukkan fungsi kendala ke-{i + 1}: ")
        label_kendala_func.pack()
        entry_kendala_func = tk.Entry(kendala_entries_frame, width=40)
        entry_kendala_func.pack()
        entry_kendala_funcs.append(entry_kendala_func)

        label_kendala_tanda = tk.Label(kendala_entries_frame, text=f"Masukkan tanda kendala ke-{i + 1} (<=, >=, =): ")
        label_kendala_tanda.pack()
        entry_kendala_tand = tk.Entry(kendala_entries_frame, width=5)
        entry_kendala_tand.pack()
        entry_kendala_tanda.append(entry_kendala_tand)

def reset_fields():
    global entry_vars, var_entries_frame, entry_kendala_funcs, entry_kendala_tanda, kendala_entries_frame

    if 'entry_vars' in globals():
        for entry in entry_vars:
            entry.delete(0, tk.END)
        entry_vars.clear()

    if 'entry_kendala_funcs' in globals():
        for entry in entry_kendala_funcs:
            entry.delete(0, tk.END)
        entry_kendala_funcs.clear()

    if 'entry_kendala_tanda' in globals():
        for entry in entry_kendala_tanda:
            entry.delete(0, tk.END)
        entry_kendala_tanda.clear()

    if 'var_entries_frame' in globals():
        var_entries_frame.destroy()

    if 'kendala_entries_frame' in globals():
        kendala_entries_frame.destroy()

root = tk.Tk()
root.title("Kondisi KKT Solver")

var_frame = tk.LabelFrame(root, text="Variabel")
var_frame.pack(padx=10, pady=10, fill=tk.BOTH)

label_variabel = tk.Label(var_frame, text="Masukkan banyak variabel:")
label_variabel.pack()

entry_variabel = tk.Entry(var_frame)
entry_variabel.pack()

button_add_var = tk.Button(var_frame, text="Tambah Variabel", command=add_variable_fields)
button_add_var.pack(pady=5)

constraint_frame = tk.LabelFrame(root, text="Kendala")
constraint_frame.pack(padx=10, pady=10, fill=tk.BOTH)

label_kendala = tk.Label(constraint_frame, text="Masukkan banyak kendala:")
label_kendala.pack()

entry_kendala = tk.Entry(constraint_frame)
entry_kendala.pack()

button_add_kendala = tk.Button(constraint_frame, text="Tambah Kendala", command=add_constraint_fields)
button_add_kendala.pack(pady=5)

fungsi_frame = tk.LabelFrame(root, text="Fungsi Objektif")
fungsi_frame.pack(padx=10, pady=10, fill=tk.BOTH)

label_fungsi = tk.Label(fungsi_frame, text="Masukkan fungsi objektif:")
label_fungsi.pack()

entry_fungsi = tk.Entry(fungsi_frame)
entry_fungsi.pack()

label_solusi = tk.Label(root, text="Apakah solusi lebih dari 0? (y/t):")
label_solusi.pack()

entry_solusi = tk.Entry(root)
entry_solusi.pack()

solve_button = tk.Button(root, text="Solve", command=display_kkt_results)
solve_button.pack(pady=10)

reset_button = tk.Button(root, text="Reset", command=reset_fields)
reset_button.pack(pady=5)

root.mainloop()